In [393]:
from pymongo import MongoClient
import pandas as pd
pd.set_option('display.max_columns', 100)
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import re
import string

[nltk_data] Downloading package wordnet to /Users/natali/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [215]:
client = MongoClient()
db     = client.fletcher
reviews   = db.reviews

In [218]:
#Select needed fields

In [241]:
cursor = reviews.find({},{ 'Accommodation': 1, 'City Life': 1, 'Clubs and Societies':1, 'Course and Lecturers':1,
                         'Facilities':1, 'Job Prospects': 1, 'OVERALL UNIVERSITY RATING': 1, 'Student Support':1,
                         "Students' Union" : 1, 'Uni Facilities' : 1, 'college' : 1, 'date' : 1})

In [242]:
df = pd.DataFrame(list(cursor))

In [277]:
cursor = reviews.find({},{ 'text_review OVERALL UNIVERSITY RATING': 1, 'college' : 1})

In [278]:
df_overall_review = pd.DataFrame(list(cursor))

In [326]:
df_overall_review.head()

,_id,college,text_review OVERALL UNIVERSITY RATING
0,5c75db20c0bfe17384b4d259,"Northumbria University, Newcastle",great and helpful experience\nprovide good tea...
1,5c75db20c0bfe17384b4d262,University of Cumbria,enjoying it. good mix of lectures and practica...
2,5c75db20c0bfe17384b4d26b,"Northumbria University, Newcastle","Sports centre is really a amazing place, Infra..."
3,5c75db20c0bfe17384b4d274,University of Hull,university has been interesting the first year...
4,5c75db20c0bfe17384b4d27d,"Northumbria University, Newcastle","I am quite enjoy my university life at here, t..."


In [288]:
text = list(df_overall_review[df_overall_review['college'] == 'University of Cumbria']['text_review OVERALL UNIVERSITY RATING'])

In [335]:
alphanumeric = lambda x: re.sub('\w*\d\w*', ' ', x)
punc_lower = lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x.lower())
letters_only = lambda X: re.sub("[^a-zA-Z]", " ", str(X))

df_overall_review['reviews'] = df_overall_review['text_review OVERALL UNIVERSITY RATING'].map(letters_only).map(alphanumeric).map(punc_lower)
df_overall_review.head()

,_id,college,text_review OVERALL UNIVERSITY RATING,reviews
0,5c75db20c0bfe17384b4d259,"Northumbria University, Newcastle",great and helpful experience\nprovide good tea...,great and helpful experience provide good teac...
1,5c75db20c0bfe17384b4d262,University of Cumbria,enjoying it. good mix of lectures and practica...,enjoying it good mix of lectures and practica...
2,5c75db20c0bfe17384b4d26b,"Northumbria University, Newcastle","Sports centre is really a amazing place, Infra...",sports centre is really a amazing place infra...
3,5c75db20c0bfe17384b4d274,University of Hull,university has been interesting the first year...,university has been interesting the first year...
4,5c75db20c0bfe17384b4d27d,"Northumbria University, Newcastle","I am quite enjoy my university life at here, t...",i am quite enjoy my university life at here t...


In [337]:
X = df_overall_review.reviews

In [345]:
cv1 = CountVectorizer(stop_words='english')

X_cv1 = cv1.fit_transform(X)

sm = pd.DataFrame(X_cv1.toarray(), columns=cv1.get_feature_names()).head()

In [380]:
# The second document-term matrix has both unigrams and bigrams, and indicators instead of counts
cv2 = CountVectorizer(stop_words='english')

X_train_cv2 = cv2.fit_transform(X)
pd.DataFrame(X_train_cv2.toarray(), columns=cv2.get_feature_names()).head()

,aa,aaa,aaaaaaa,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaand,aaaaaaaaaassssssaaaaaaa,aaahhh,aacsb,aad,aam,aanas,aany,aarhus,ab,abandoned,abb,abbey,abble,abbreviated,abchance,abd,abdbtge,aber,aberarchers,aberbsl,aberdeen,abernathy,abers,abertay,aberworks,aberysteyth,aberystwtyth,aberystwyth,aberyswyth,aberywtsyth,abet,abetter,abhorrent,abilities,ability,abit,abject,abkut,able,abled,ableton,abload,...,youll,youn,young,younger,youngest,youngnpeople,youngsters,youre,yourpreferred,youth,youthful,youtube,youtubed,youv,youve,yoy,ypd,ypu,yr,yrs,ysgol,ysgolion,ysj,ysjfamily,ysju,ythat,ytyried,yukon,yum,yummy,yup,yupp,yytttt,zambia,zapping,zaps,zealand,zeitgeist,zero,zillion,zing,zith,zombies,zone,zones,zoo,zoology,zoos,zouk,zumba
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [381]:
document = cv2.get_feature_names()

In [389]:
stemmer = SnowballStemmer('english')
new_corpus=[' '.join([stemmer.stem(word) for word in text.split(' ')])
          for text in X]

In [391]:
lemmer=WordNetLemmatizer()
new_corpus=[' '.join([lemmer.lemmatize(word) for word in text.split(' ')])
          for text in new_corpus]

In [394]:
new_corpus

['great and help experi provid good teach stuff to support the student and help them to find a suitabl cours   good room and facil to studi in it ',
 'enjoy it  good mix of lectur and practic session and placement   can be hard balanc all the assign a this uni give you more than other  person tutor are veri help',
 'sport centr is realli a amaz place  infrastructur is mind blow  librari ha    hr access  food and coffe price could be better  tuition fee can be reduc for intern student and provid more scholarship ',
 'univers ha been interest the first year wa hard a the hall wa awkward and it wa hard to find peopl you got along with  it got better in second year howev due to outsid problem to uni it got hard but the union wa there ',
 'i am quit enjoy my univers life at here  tutor and staff are help  howev  the fee of the sport are too expens which i think is the biggest problem that uni didn t realli concern the balanc between student health and financ ',
 'it form one of the best par

In [382]:
stemmer = SnowballStemmer('english')
lemmer=WordNetLemmatizer()
new_corpus=[' '.join([lemmer.lemmatize(stemmer.stem(word)) for word in document])]

In [387]:
X_train_cv2 = cv2.fit_transform(new_corpus)

In [384]:
pd.DataFrame(X_cv1.toarray(), columns=new_corpus).head()

ValueError: Shape of passed values is (18835, 86430), indices imply (1, 86430)

In [248]:
df = df.fillna(df.mean())

In [256]:
df.to_csv('Data/for_violen.csv')

In [249]:
df_cat = df.groupby('college').mean().reset_index()

In [240]:
df_cat.to_csv('Data/comparison.csv')

### Aggregation

In [188]:
#The same thing with aggregation in mongoDb

In [141]:
aggr = reviews.aggregate([ 
                     { '$group': { '_id': '$college', 'Overall': { '$avg': "$OVERALL UNIVERSITY RATING" },
                                 
                                 '_id': '$college', 'Carrer': { '$avg': '$Job Prospects' },
                                 '_id': '$college', 'Lectures': { '$avg': '$Course and Lecturers' }          
                     }           
                     }
                   ])

In [142]:
df2 = pd.DataFrame(list(aggr))

In [144]:
df2

,Carrer,Lectures,Overall,_id
0,5.000000,5.000000,5.000000,London School of Planning and Management
1,NaN,4.000000,4.000000,CU Coventry
2,5.000000,1.000000,5.000000,University Centre West Anglia
3,5.000000,5.000000,5.000000,East Riding College
4,NaN,4.438596,4.505882,Bournemouth and Poole College
5,NaN,3.631579,3.210526,Kingston College
6,5.000000,4.900000,5.000000,Al-Maktoum College of Higher Education
7,NaN,4.275862,4.275862,Walsall College
8,NaN,4.535714,4.607143,UK College of Business and Computing
9,4.928571,4.785714,4.857143,Newcastle College University Centre


In [64]:
df1['OVERALL UNIVERSITY RATING']

KeyError: 'OVERALL UNIVERSITY RATING'

In [62]:
df1.columns

Index(['OVERALL UNIVERSITY RATING', '_id'], dtype='object')

In [49]:
for i in cursor:

    print(i)

{'_id': ObjectId('5c74fd32c0bfe164873efbc9'), 'OVERALL UNIVERSITY RATING': 3}
{'_id': 2, 'OVERALL UNIVERSITY RATING': 3}
{'_id': ObjectId('507f191e810c19729de860ea'), 'OVERALL UNIVERSITY RATING': 3}
{'_id': ObjectId('5c7504b9c0bfe17384acb30b'), 'OVERALL UNIVERSITY RATING': 3}
{'_id': ObjectId('5c750564c0bfe17384acb30d'), 'OVERALL UNIVERSITY RATING': 3}
{'_id': ObjectId('5c755f07c0bfe17384acb310'), 'OVERALL UNIVERSITY RATING': 3}
{'_id': ObjectId('5c755f07c0bfe17384acb311'), 'OVERALL UNIVERSITY RATING': 3}
{'_id': ObjectId('5c755f07c0bfe17384acb312'), 'OVERALL UNIVERSITY RATING': 3}
{'_id': ObjectId('5c755f07c0bfe17384acb313'), 'OVERALL UNIVERSITY RATING': 3}
{'_id': ObjectId('5c755f07c0bfe17384acb314'), 'OVERALL UNIVERSITY RATING': 3}
{'_id': ObjectId('5c755f07c0bfe17384acb315'), 'OVERALL UNIVERSITY RATING': 3}
{'_id': ObjectId('5c755f07c0bfe17384acb316'), 'OVERALL UNIVERSITY RATING': 3}
{'_id': ObjectId('5c755f07c0bfe17384acb317'), 'OVERALL UNIVERSITY RATING': 3}
{'_id': ObjectId('5c7

{'_id': ObjectId('5c75819ac0bfe17384afec13'), 'OVERALL UNIVERSITY RATING': 3}
{'_id': ObjectId('5c75819ac0bfe17384afec14'), 'OVERALL UNIVERSITY RATING': 3}
{'_id': ObjectId('5c75819ac0bfe17384afec15'), 'OVERALL UNIVERSITY RATING': 3}
{'_id': ObjectId('5c75819ac0bfe17384afec16'), 'OVERALL UNIVERSITY RATING': 3}
{'_id': ObjectId('5c75819ac0bfe17384afec20'), 'OVERALL UNIVERSITY RATING': 5}
{'_id': ObjectId('5c75819ac0bfe17384afec21'), 'OVERALL UNIVERSITY RATING': 5}
{'_id': ObjectId('5c75819ac0bfe17384afec22'), 'OVERALL UNIVERSITY RATING': 5}
{'_id': ObjectId('5c75819ac0bfe17384afec23'), 'OVERALL UNIVERSITY RATING': 5}
{'_id': ObjectId('5c75819ac0bfe17384afec24'), 'OVERALL UNIVERSITY RATING': 5}
{'_id': ObjectId('5c75819ac0bfe17384afec25'), 'OVERALL UNIVERSITY RATING': 5}
{'_id': ObjectId('5c75819ac0bfe17384afec26'), 'OVERALL UNIVERSITY RATING': 5}
{'_id': ObjectId('5c75819ac0bfe17384afec27'), 'OVERALL UNIVERSITY RATING': 5}
{'_id': ObjectId('5c75819ac0bfe17384afec28'), 'OVERALL UNIVERSIT

{'_id': ObjectId('5c758341c0bfe17384b024a5'), 'OVERALL UNIVERSITY RATING': 5}
{'_id': ObjectId('5c758341c0bfe17384b024a6'), 'OVERALL UNIVERSITY RATING': 5}
{'_id': ObjectId('5c758341c0bfe17384b024a7'), 'OVERALL UNIVERSITY RATING': 5}
{'_id': ObjectId('5c758341c0bfe17384b024a8'), 'OVERALL UNIVERSITY RATING': 5}
{'_id': ObjectId('5c758341c0bfe17384b024a9'), 'OVERALL UNIVERSITY RATING': 5}
{'_id': ObjectId('5c758341c0bfe17384b024aa'), 'OVERALL UNIVERSITY RATING': 5}
{'_id': ObjectId('5c758341c0bfe17384b024ab'), 'OVERALL UNIVERSITY RATING': 5}
{'_id': ObjectId('5c758341c0bfe17384b024ac'), 'OVERALL UNIVERSITY RATING': 5}
{'_id': ObjectId('5c758341c0bfe17384b024ad'), 'OVERALL UNIVERSITY RATING': 5}
{'_id': ObjectId('5c758343c0bfe17384b024ae'), 'OVERALL UNIVERSITY RATING': 2}
{'_id': ObjectId('5c758343c0bfe17384b024af'), 'OVERALL UNIVERSITY RATING': 2}
{'_id': ObjectId('5c758343c0bfe17384b024b0'), 'OVERALL UNIVERSITY RATING': 2}
{'_id': ObjectId('5c758343c0bfe17384b024b1'), 'OVERALL UNIVERSIT

{'_id': ObjectId('5c759f44c0bfe17384b3fcc1'), 'OVERALL UNIVERSITY RATING': 5}
{'_id': ObjectId('5c759f44c0bfe17384b3fcc2'), 'OVERALL UNIVERSITY RATING': 5}
{'_id': ObjectId('5c759f44c0bfe17384b3fcc3'), 'OVERALL UNIVERSITY RATING': 5}
{'_id': ObjectId('5c759f44c0bfe17384b3fcc4'), 'OVERALL UNIVERSITY RATING': 5}
{'_id': ObjectId('5c759f44c0bfe17384b3fcc5'), 'OVERALL UNIVERSITY RATING': 5}
{'_id': ObjectId('5c759f44c0bfe17384b3fcd0'), 'OVERALL UNIVERSITY RATING': 5}
{'_id': ObjectId('5c759f44c0bfe17384b3fcd1'), 'OVERALL UNIVERSITY RATING': 5}
{'_id': ObjectId('5c759f44c0bfe17384b3fcd2'), 'OVERALL UNIVERSITY RATING': 5}
{'_id': ObjectId('5c759f44c0bfe17384b3fcd3'), 'OVERALL UNIVERSITY RATING': 5}
{'_id': ObjectId('5c759f44c0bfe17384b3fcd4'), 'OVERALL UNIVERSITY RATING': 5}
{'_id': ObjectId('5c759f44c0bfe17384b3fcd5'), 'OVERALL UNIVERSITY RATING': 5}
{'_id': ObjectId('5c759f44c0bfe17384b3fcd6'), 'OVERALL UNIVERSITY RATING': 5}
{'_id': ObjectId('5c759f44c0bfe17384b3fcd7'), 'OVERALL UNIVERSIT

{'_id': ObjectId('5c759fccc0bfe17384b41046'), 'OVERALL UNIVERSITY RATING': 3}
{'_id': ObjectId('5c759fccc0bfe17384b41047'), 'OVERALL UNIVERSITY RATING': 3}
{'_id': ObjectId('5c759fccc0bfe17384b41048'), 'OVERALL UNIVERSITY RATING': 3}
{'_id': ObjectId('5c759fccc0bfe17384b41049'), 'OVERALL UNIVERSITY RATING': 3}
{'_id': ObjectId('5c759fccc0bfe17384b4104a'), 'OVERALL UNIVERSITY RATING': 3}
{'_id': ObjectId('5c759fccc0bfe17384b4104b'), 'OVERALL UNIVERSITY RATING': 3}
{'_id': ObjectId('5c759fccc0bfe17384b4104c'), 'OVERALL UNIVERSITY RATING': 3}
{'_id': ObjectId('5c759fccc0bfe17384b4104d'), 'OVERALL UNIVERSITY RATING': 3}
{'_id': ObjectId('5c759fccc0bfe17384b4104e'), 'OVERALL UNIVERSITY RATING': 5}
{'_id': ObjectId('5c759fccc0bfe17384b4104f'), 'OVERALL UNIVERSITY RATING': 5}
{'_id': ObjectId('5c759fccc0bfe17384b41050'), 'OVERALL UNIVERSITY RATING': 5}
{'_id': ObjectId('5c759fccc0bfe17384b41051'), 'OVERALL UNIVERSITY RATING': 5}
{'_id': ObjectId('5c759fccc0bfe17384b41052'), 'OVERALL UNIVERSIT

In [36]:
cursor = reviews.find({}, {'college': 'Northumbria University, Newcastle'})
for i in cursor:
    print(i)

KeyError: 'OVERALL UNIVERSITY RATING'

In [35]:
cursor = reviews.find({}, {'college': 'Northumbria University, Newcastle'})
overall = []
for review in cursor:
    overall.append(review['OVERALL UNIVERSITY RATING'])
    
    

#list(cursor)

KeyError: 'OVERALL UNIVERSITY RATING'

In [34]:
overall

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [ ]:
for i in reviews.find():
    print(i['_id']);